# PyFluent Simulation Mode Guide

This notebook demonstrates how to use PyFluent in simulation mode to run protocols on Tecan FluentControl liquid handlers.

## Overview

Simulation mode allows you to:
- Test protocols without physical hardware
- Visualize pipetting operations in the 3D viewer
- Debug and develop protocols safely
- Verify labware and liquid class configurations

## Prerequisites

1. **FluentControl installed** - Tecan FluentControl software must be installed
2. **API Channel Method** - Create a method in FluentControl with an "API Channel" action
3. **Worktable configured** - Set up labware on the worktable/deck
4. **Liquid classes configured** - Ensure liquid classes are configured for your tip types


## Step 1: Import and Initialize

First, import PyFluent and set up the backend for simulation mode.


In [ ]:
import sys
import os
import asyncio

# CRITICAL: Set coinit_flags BEFORE importing comtypes
if not hasattr(sys, 'coinit_flags'):
    sys.coinit_flags = 0

try:
    import comtypes.client
except ImportError:
    pass

# Add PyFluent to path (adjust if needed)
sys.path.insert(0, os.path.dirname(os.path.dirname(os.path.abspath(''))))

from pyfluent.backends.fluent_visionx import FluentVisionX
from pyfluent.backends.inspector import print_configuration_summary

print("PyFluent imported successfully!")


## Step 2: Create Backend Instance

Create a FluentVisionX backend with `simulation_mode=True`. This will start FluentControl in simulation mode.


In [ ]:
# Create backend for simulation mode
backend = FluentVisionX(
    num_channels=8,  # Number of pipetting channels (typically 8 for Fluent)
    simulation_mode=True,  # Enable simulation mode
    with_visualization=False  # Use FluentControl's 3D viewer, not PyLabRobot's
)

print("Backend created for simulation mode")


## Step 3: Connect to FluentControl

Connect to FluentControl. This will start FluentControl in simulation mode if it's not already running.


In [ ]:
async def connect():
    """Connect to FluentControl."""
    print("Connecting to FluentControl...")
    await backend.setup()
    print("✓ Connected to FluentControl")
    
    # Enable 3D viewer and animation
    print("\nEnabling 3D viewer...")
    backend.show_3d_viewer()
    backend.enable_animation(True)
    backend.set_simulation_speed(1.0)  # Normal speed
    print("✓ 3D viewer enabled")
    
    return backend

# Run the async function
backend = await connect()


## Step 4: Check Available Configuration

Before running a protocol, it's helpful to verify what labware and liquid classes are available in your FluentControl setup.


In [ ]:
# Print available labware and liquid classes
print_configuration_summary(backend)


**Note:** If the API doesn't provide labware/liquid class lists, check FluentControl's worktable/deck view and liquid class settings manually.

Common labware names:
- `96 Well Flat[001]`
- `96 Well Flat[002]`
- `MCA Thru Deck Waste Chute with Tip Drop Guide_2` (for waste)

Common liquid classes:
- `DMSO Free Single_1`
- `Water Test No Detect`
- `Empty Tip`


## Step 5: Start API Channel Method

To send commands via the API, you need a FluentControl method with an "API Channel" action running. This method keeps the API channel open.


In [ ]:
async def start_api_channel(method_name="demo"):
    """Start a method with API Channel action."""
    print(f"Starting method '{method_name}'...")
    print("Note: This method should contain an 'API Channel' action")
    
    # Run method (wait_for_completion=False keeps it running)
    success = await backend.run_method(method_name, wait_for_completion=False)
    print(f"✓ Method started: {success}")
    
    # Wait for API channel to open
    print("\nWaiting for API channel to open...")
    channel_ready = await backend.wait_for_channel(timeout=90)
    
    if channel_ready:
        print("✓ API channel is ready!")
        return True
    else:
        print("✗ API channel did not open")
        return False

# Start the API channel
channel_ready = await start_api_channel("demo")


## Step 6: Basic Operations

Now you can send pipetting commands. Here are the basic operations:


### 6.1 Pick Up Tips

Pick up tips from a tip rack.


In [ ]:
# Pick up tips
print("Picking up tips...")
backend.get_tips(
    diti_type="TOOLTYPE:LiHa.TecanDiTi/TOOLNAME:FCA, 200ul",  # Tip type
    tip_indices=[0]  # Use tip 0 (first tip), or None for all 8 tips
)
print("✓ Tips picked up!")

# Wait a moment for movement to complete in simulation
await asyncio.sleep(2)


### 6.2 Aspirate Liquid

Aspirate liquid from a well plate.


In [ ]:
# Aspirate from well A1 (offset 0)
source_labware = "96 Well Flat[001]"  # Change to match your worktable
liquid_class = "DMSO Free Single_1"  # Change to match your liquid class

print(f"Aspirating 50µL from {source_labware} well A1...")
backend.aspirate_volume(
    volumes=[50],  # 50µL
    labware=source_labware,
    liquid_class=liquid_class,
    well_offsets=[0],  # Well A1 (offset 0)
    tip_indices=[0]  # Use tip 0
)
print("✓ Aspirated successfully!")

await asyncio.sleep(2)


### 6.3 Dispense Liquid

Dispense liquid to a well plate.


In [ ]:
# Dispense to well A2 (offset 1)
dest_labware = "96 Well Flat[001]"  # Can be same or different plate

print(f"Dispensing 50µL to {dest_labware} well A2...")
backend.dispense_volume(
    volumes=[50],  # 50µL
    labware=dest_labware,
    liquid_class=liquid_class,  # Use same liquid class
    well_offsets=[1],  # Well A2 (offset 1)
    tip_indices=[0]  # Use tip 0
)
print("✓ Dispensed successfully!")

await asyncio.sleep(2)


### 6.4 Drop Tips

Drop tips to waste.


In [ ]:
# Drop tips to waste
waste_location = "MCA Thru Deck Waste Chute with Tip Drop Guide_2"  # Adjust if needed

print(f"Dropping tips to {waste_location}...")
backend.drop_tips_to_location(
    labware=waste_location,
    tip_indices=[0]  # Use tip 0, or None for all tips
)
print("✓ Tips dropped!")

await asyncio.sleep(2)


## Step 7: Multi-Channel Operations

You can use multiple channels simultaneously for parallel pipetting.


In [ ]:
# Pick up all 8 tips
print("Picking up all 8 tips...")
backend.get_tips(
    diti_type="TOOLTYPE:LiHa.TecanDiTi/TOOLNAME:FCA, 200ul",
    tip_indices=None  # None = all 8 tips
)
await asyncio.sleep(2)

# Aspirate from multiple wells (column 1: A1-H1)
print("Aspirating from column 1 (A1-H1)...")
backend.aspirate_volume(
    volumes=[50] * 8,  # 50µL for each tip
    labware=source_labware,
    liquid_class=liquid_class,
    well_offsets=[0, 1, 2, 3, 4, 5, 6, 7],  # A1-H1 (offsets 0-7)
    tip_indices=[0, 1, 2, 3, 4, 5, 6, 7]  # All 8 tips
)
await asyncio.sleep(2)

# Dispense to multiple wells (column 2: A2-H2)
print("Dispensing to column 2 (A2-H2)...")
backend.dispense_volume(
    volumes=[50] * 8,
    labware=dest_labware,
    liquid_class=liquid_class,
    well_offsets=[12, 13, 14, 15, 16, 17, 18, 19],  # A2-H2 (offsets 12-19)
    tip_indices=[0, 1, 2, 3, 4, 5, 6, 7]
)
await asyncio.sleep(2)

# Drop all tips
print("Dropping all tips...")
backend.drop_tips_to_location(
    labware=waste_location,
    tip_indices=None  # All tips
)
print("✓ Multi-channel operations complete!")


## Step 8: Complete Example Protocol

Here's a complete example that combines all operations:


In [ ]:
async def run_complete_protocol():
    """Run a complete protocol example."""
    
    # Configuration
    method_name = "demo"  # Your API Channel method name
    source_plate = "96 Well Flat[001]"  # Source plate
    dest_plate = "96 Well Flat[001]"  # Destination plate
    liquid_class = "DMSO Free Single_1"  # Your liquid class
    waste_location = "MCA Thru Deck Waste Chute with Tip Drop Guide_2"
    
    try:
        # 1. Connect
        print("=" * 60)
        print("PyFluent Simulation Protocol")
        print("=" * 60)
        print("\n1. Connecting to FluentControl...")
        await backend.setup()
        backend.show_3d_viewer()
        backend.enable_animation(True)
        backend.set_simulation_speed(1.0)
        print("✓ Connected")
        
        # 2. Start API channel
        print("\n2. Starting API channel method...")
        await backend.run_method(method_name, wait_for_completion=False)
        channel_ready = await backend.wait_for_channel(timeout=90)
        if not channel_ready:
            print("✗ API channel failed")
            return
        print("✓ API channel ready")
        
        # 3. Pick up tips
        print("\n3. Picking up tips...")
        backend.get_tips(diti_type="TOOLTYPE:LiHa.TecanDiTi/TOOLNAME:FCA, 200ul", tip_indices=[0])
        await asyncio.sleep(2)
        
        # 4. Aspirate from A1
        print("\n4. Aspirating 50µL from well A1...")
        backend.aspirate_volume(
            volumes=[50],
            labware=source_plate,
            liquid_class=liquid_class,
            well_offsets=[0],
            tip_indices=[0]
        )
        await asyncio.sleep(2)
        
        # 5. Dispense to A2
        print("\n5. Dispensing 50µL to well A2...")
        backend.dispense_volume(
            volumes=[50],
            labware=dest_plate,
            liquid_class=liquid_class,
            well_offsets=[1],
            tip_indices=[0]
        )
        await asyncio.sleep(2)
        
        # 6. Drop tips
        print("\n6. Dropping tips...")
        backend.drop_tips_to_location(labware=waste_location, tip_indices=[0])
        await asyncio.sleep(2)
        
        print("\n" + "=" * 60)
        print("✓ Protocol complete!")
        print("=" * 60)
        
    except Exception as e:
        print(f"\n✗ Error: {e}")
        import traceback
        traceback.print_exc()
    finally:
        # Cleanup
        print("\nCleaning up...")
        await backend.stop()
        print("✓ Done")

# Run the complete protocol
await run_complete_protocol()


## Troubleshooting

### Common Issues

1. **"Select a valid labware" error**
   - Check the labware name in FluentControl worktable
   - Names are case-sensitive and must match exactly
   - Use `print_configuration_summary()` to see available labware

2. **"Liquid subclass missing" error**
   - Check liquid class name in FluentControl
   - Ensure the liquid class is configured for your tip type
   - Liquid class names are case-sensitive

3. **API channel doesn't open**
   - Ensure your method contains an "API Channel" action
   - Method should stay running (add a loop or wait action)
   - Check FluentControl for method errors

4. **Recovery mode errors**
   - PyFluent automatically clears recovery information
   - If issues persist, manually clear the last run in FluentControl

5. **No movement in 3D viewer**
   - Ensure `show_3d_viewer()` was called
   - Check that animation is enabled: `enable_animation(True)`
   - Verify simulation speed: `set_simulation_speed(1.0)`

## Next Steps

- Explore more complex protocols with multiple plates
- Use the Protocol class for structured protocol definitions
- Integrate with PyLabRobot for advanced resource management
- Check the PyFluent documentation for more features
